In [61]:
import pandas as pd
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/thais/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
from nltk.tokenize import word_tokenize

In [27]:
def tokenize_text(text):
    inside = False
    start = False
    text_lines_mod = []
    
    text_lines = word_tokenize(text)
    
    for line in text_lines:
        if "B__B-" in line:
            start = True
            inside = True
            token = line.split("B__B")[1].split("-")[1]
        elif "E__E-" in line:
            inside = False
        else:
            if not inside:
                line = " O"
            else:
                if start:
                    line = " B-" + token
                else:
                    line = " I-" + token
                start = False

            text_lines_mod.append(line)
            
    return text_lines_mod

In [28]:
def get_tokenized_text_lines(base_df, base_text_df):
    marked_texts = []

    for arquivo in base_df["arquivo_rast"].unique():
        entidades_df = base_df[base_df["arquivo_rast"] == arquivo].copy().reset_index(drop=True)[["entidade", "ner_token", "loc_begin", "loc_end", "arquivo_rast"]]
        text = base_text_df[base_text_df["arquivo_rast"] == arquivo]["text"].iloc[0]

        entidades_df = entidades_df.sort_values(["loc_end"], ascending=False).reset_index(drop=True)

        for token, begin, end in entidades_df[["ner_token", "loc_begin", "loc_end"]].itertuples(index=False):
            mark_token_begin = " B__B-" + token + "-B__B "
            mark_token_end = " E__E-" + token + "-E__E "

            text = text[:end] + mark_token_end +  text[end:]
            text = text[:begin] + mark_token_begin +  text[begin:]

        marked_texts.append(text)
        
    marked_texts_df = pd.DataFrame(pd.Series(marked_texts), columns=["marked_text"])
    marked_texts_df["arquivo_rast"] = base_df["arquivo_rast"].unique()
    
    #return marked_texts_df

    marked_texts_df["treated_text"] = clean_text_dodfs(marked_texts_df["marked_text"])
    marked_texts_df["treated_text"] = marked_texts_df.apply(lambda row: clean_text_by_word(row["treated_text"]), axis=1)
    
    marked_texts_df["tokenized_text"] = marked_texts_df.apply(lambda row: tokenize_text(row["treated_text"]), axis=1)
    
    return marked_texts_df

In [29]:
def get_tokenized_text_lines_geral(base_df, base_text_df):
    marked_texts = []

    for arquivo in base_df["arquivo_rast"].unique():
        entidades_df = base_df[base_df["arquivo_rast"] == arquivo].copy().reset_index(drop=True)[["entidade", "ner_token", "loc_begin", "loc_end", "arquivo_rast"]]
        text = base_text_df[base_text_df["arquivo_rast"] == arquivo]["text"].iloc[0]

        entidades_df = entidades_df.sort_values(["loc_end"], ascending=False).reset_index(drop=True)

        for token, begin, end in entidades_df[["ner_token", "loc_begin", "loc_end"]].itertuples(index=False):
            mark_token_begin = " B__B-" + token + "-B__B "
            mark_token_end = " E__E-" + token + "-E__E "

            text = text[:end] + mark_token_end +  text[end:]
            text = text[:begin] + mark_token_begin +  text[begin:]

        marked_texts.append(text)
        
    marked_texts_df = pd.DataFrame(pd.Series(marked_texts), columns=["marked_text"])
    marked_texts_df["arquivo_rast"] = base_df["arquivo_rast"].unique()
    
    #return marked_texts_df

    marked_texts_df["treated_text"] = clean_text_dodfs(marked_texts_df["marked_text"])
    marked_texts_df["treated_text"] = marked_texts_df.apply(lambda row: clean_text_by_word(row["treated_text"]), axis=1)
    
    tokenized_texts = marked_texts_df.apply(lambda row: [tokenize_text(row["treated_text"]), row["arquivo_rast"]], axis=1)
    
    return tokenized_texts

In [30]:
def clean_text_dodfs(dodfs_texts):

    start_page_patterns = [r"\nPÁGINA\s([0-9]{1,5})", r"\nDIÁRIO\sOFICIAL\sDO\sDISTRITO\sFEDERAL",
                           r"\nNº(.+?)2([0-9]{3})", r"\nxx([a-z]{0,10}) Diário Oficial do Distrito Federal xx([a-z]{0,10})",
                           r"\nDiário Oficial do Distrito Federal"]

    end_page_patterns = [r"Documento assinado digitalmente conforme MP nº 2.200-2 de 24/08/2001, que institui a",
                            r"Infraestrutura de Chaves Públicas Brasileira ICP-Brasil",
                            r"Este documento pode ser verificado no endereço eletrônico",
                            r"http://wwwin.gov.br/autenticidade.html",
                            r"pelo código ([0-9]{15,18})",
                            r"\nDocumento assinado digitalmente, original em https://www.dodf.df.gov.br",
                        r"http:/Awwwin.gov.br/autenticidade.html",
                        r"Documento assinado digitalmente conforme MP nº 2.200-2 de 24/08/2001,",
                        r"\nque institui a\n",
                        r"\nhttp://www.in.gov.br/autenticidade.html",
                        r"\nhttp://www.in.gov.brautenticidade html",
                        r"Documento assinado digitalmente conforme MP n 2.200-2 de 24/08/2001, que institui a .",
                        r"http://www.in.gov.brautenticidade html,"]

    middle_page_patterns = [r"xx([a-z]{1,10}) ", r" xx([a-z]{1,10})", r"xx([a-z]{1,10})"]
    
    special_middle_page_patterns = [r"\n-\n", r"\n- -\n", r"\n- - -\n", r"\n[\.\,\-\—]\n", r"— -", r". -",
                                   r"\r[\.\,\-\—]\r", r"\n-\r", r"\r"]
    
    start_page_patterns = "|".join(start_page_patterns)
    middle_page_patterns = "|".join(middle_page_patterns)
    end_page_patterns = "|".join(end_page_patterns)
    
    page_patterns = [start_page_patterns, middle_page_patterns, end_page_patterns]
    page_patterns = "|".join(page_patterns)
    
    return dodfs_texts.str.replace(page_patterns, "", regex=True).replace(special_middle_page_patterns, "\n", regex=True)

In [31]:
def clean_text_by_word(text):
    a = "\n".join([l for l in text.split("\n") if l != ""])
    words = a.replace("\n", " ").split(" ")
    words = [w for w in words if w != ""]
    
    m_words = []
    dash_cut = False

    for i in range(len(words)):
        word = words[i]

        if (word[-1] == "-") and (i+1)<len(words):
            word = word[:-1] + words[i+1]
            i += 1

        m_words.append(word)
        
    return " ".join(m_words)

In [32]:
def specific_parse(base_df, func=None):
    if func == None:
        return base_df
    else:
        return func(base_df)

In [33]:
# Transformação dos rótulos dados no LabelBox para os rótulos que serão usados no modelo NER

ner_tokens_extrato_contrato = {
    "Número do ajuste": "NUM_AJUSTE",
    "Órgão contratante": "CONTRATANTE",
    "Entidade contratada": "CONTRATADA",
    "Entidades convenentes": "CONVENENTE",
    "Processo do GDF": "PROCESSO",
    "Objeto do ajuste": "OBJ_AJUSTE",
    "Data de assinatura do ajuste": "DATA_ASSINATURA",
    "Vigência do ajuste": "VIGENCIA",
    "Valor do ajuste": "VALOR",
    "Código da unidade orçamentária": "CODIGO_UO",
    "Programa de trabalho": "PT",
    "Natureza da despesa": "ND",
    "Nota de empenho": "NE",
    "Órgão/entidade licitante": "ORGAO_LICITANTE",
    "Número da licitação que gerou o contrato": "NUM_LICITACAO",
    "Órgão gerenciador da ata": "OG_ATA",
    "Identificação de dispensa": "IDENT_DISPENSA",
    "Fundamento legal da dispensa ou inexigibilidade": "FUND_DISPENSA"
}

ner_tokens_aditamento_contratual = {
    "Órgão contratante": "CONTRATANTE",
    "Número do contrato": "NUM_CONTRATO",
    "Número do termo aditivo": "NUM_ADITIVO",
    "Objeto do aditamento contratual": "OBJ_ADITIVO"
}

ner_tokens_aviso_licitacao = {
    "Órgão/entidade licitante": "ORGAO_LICITANTE",
    "Número da licitação": "NUM_LICITACAO",
    "Objeto da licitação": "OBJ_LICITACAO",
    "Modalidade de licitação": "MODALIDADE_LICITACAO",
    "Processo do GDF": "PROCESSO",
    "Valor estimado da contratação": "VALOR_ESTIMADO",
    "Data de abertura da licitação": "DATA_ABERTURA",
    "Sistema de compras utilizado": "SISTEMA_COMPRAS",
    "Código da licitação no sistema de compras utilizado": "CODIGO_SISTEMA_COMPRAS",
    "Tipo de objeto": "TIPO_OBJ"
}

ner_tokens_aviso_revogacao = {
    "Órgão/entidade licitante": "ORGAO_LICITANTE",
    "Número da licitação": "NUM_LICITACAO",
    "Modalidade de licitação": "MODALIDADE_LICITACAO",
    "Identificação revogação/anulação": "IDENT_REVOGACAO_ANULACAO"
}

ner_tokens_aviso_suspensao_licitacao = {
    "Órgão/entidade licitante": "ORGAO_LICITANTE",
    "Número da licitação": "NUM_LICITACAO",
    "Modalidade de licitação": "MODALIDADE_LICITACAO",
    "Prazo da suspensão": "PRAZO_SUSPENSAO",
    "Decisão do TCDF que determinou a suspensão": "DECISAO_TCDF"
}

In [76]:
dataset = "contratos_filtrados_v2"
path_data = "./parquets_labelbox_data/"
base_path_samples = "../atos_revisados/samples_3/"

In [39]:
os.mkdir(base_path_samples + dataset)
path_samples = base_path_samples + dataset + "/"

FileNotFoundError: [Errno 2] No such file or directory: '../atos_revisados/samples_3/contratos_filtrados_v2'

In [43]:
path_samples = base_path_samples + dataset

In [44]:
atos_data = []
atos_data_text = []

for arquivo in os.listdir(path_data):
    if ".parquet" in arquivo:
        file_path = path_data + arquivo
        file_df = pd.read_parquet(file_path)

        if "data_text" in arquivo:
            atos_data_text.append(file_df)
        else:
            atos_data.append(file_df)

In [45]:
atos_data_df = pd.concat(atos_data, ignore_index=True)
atos_data_text_df = pd.concat(atos_data_text, ignore_index=True)

In [46]:
# Limpar texto
atos_data_text_df["treated_text"] = clean_text_dodfs(atos_data_text_df["text"])
atos_data_text_df["treated_text"] = atos_data_text_df.apply(lambda row: clean_text_by_word(row["treated_text"]), axis=1)

In [47]:
print(atos_data_text_df.iloc[3]["text"])

xxbcet AVISO DE LICITAÇÃO xxecet
xxbcet PREGÃO ELETRÔNICO (SRP) Nº 02/2021 xxecet
xxbcet (AMPLA CONCORRÊNCIA) xxecet
O Distrito Federal, representado pela Secretaria de Estado da A gricultura, A bastecimento
e Desenvolvimento Rural SEAGRI/DF toma público que realizará licitação do tipo
-
xxeob

xxbob
MENOR PREÇO, na modalidade de PREGÃO na forma ELETRÔNICA, por meio de
Sistema de Registro de Preços, com previsão de abertura do certame para 21/01/2021, às
09:30 HS, Processo nº 00070-00004150/2020-93 (SEI). OBJETO: A presente licitação
tem como objeto, Registro de Preços para a eventual aquisição de tratores agrícolas de no
mínimo 120 e 140 cvs a fim de atender a Secretaria de Estado da Agricultura,
Abastecimento e Desenvolvimento Rural SEAGRI, de acordo com o detalhamento
xxbcet - xxecet
descrito no item 3 do Termo de Referência, Anexo I do Edital e demais obrigações e
informações constantes dos Anexos do Edital, com valor Total estimado de R$
1.346.049,80 (um milhão, trezentos e quaren

In [48]:
print(atos_data_text_df.iloc[3]["treated_text"])

AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO (SRP) Nº 02/2021 (AMPLA CONCORRÊNCIA) O Distrito Federal, representado pela Secretaria de Estado da A gricultura, A bastecimento e Desenvolvimento Rural SEAGRI/DF toma público que realizará licitação do tipo MENOR PREÇO, na modalidade de PREGÃO na forma ELETRÔNICA, por meio de Sistema de Registro de Preços, com previsão de abertura do certame para 21/01/2021, às 09:30 HS, Processo nº 00070-00004150/2020-93 (SEI). OBJETO: A presente licitação tem como objeto, Registro de Preços para a eventual aquisição de tratores agrícolas de no mínimo 120 e 140 cvs a fim de atender a Secretaria de Estado da Agricultura, Abastecimento e Desenvolvimento Rural SEAGRI, de acordo com o detalhamento descrito no item 3 do Termo de Referência, Anexo I do Edital e demais obrigações e informações constantes dos Anexos do Edital, com valor Total estimado de R$ 1.346.049,80 (um milhão, trezentos e quarenta e seis mil quarenta e nove reais e oitenta centavos). O Edital poderá 

In [49]:
# Dispensa de inexigibilidade e dispensa de licitação podem ser agrupados em um único rótulo de identificação
atos_data_df["titulo_entidade"] = atos_data_df["titulo_entidade"].replace("Dispensa de inexigibilidade", "Identificação de dispensa")
atos_data_df["titulo_entidade"] = atos_data_df["titulo_entidade"].replace("Dispensa de licitação", "Identificação de dispensa")

In [50]:
rep_atos = {
    "extrato_de_aditamento_contratual": "aviso_de_aditamento_contratual",
    "aviso_de_anulacao_e_revogacao": "aviso_de_revogacao_anulacao_de_licitacao"
}

atos_data_df["ato"] = atos_data_df["ato"].replace(rep_atos)
atos_data_text_df["ato"] = atos_data_text_df["ato"].replace(rep_atos)

In [51]:
atos_data_df["ner_token"] = ""

In [52]:
atos_data_df.loc[atos_data_df["ato"].isin(["extrato_de_contrato_ou_convenio"]), "ner_token"] = atos_data_df.loc[atos_data_df["ato"].isin(["extrato_de_contrato_ou_convenio"]), "titulo_entidade"].replace(ner_tokens_extrato_contrato)
atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_aditamento_contratual"]), "ner_token"] = atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_aditamento_contratual"]), "titulo_entidade"].replace(ner_tokens_aditamento_contratual)
atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_licitacao"]), "ner_token"] = atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_licitacao"]), "titulo_entidade"].replace(ner_tokens_aviso_licitacao)
atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_revogacao_anulacao_de_licitacao"]), "ner_token"] = atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_revogacao_anulacao_de_licitacao"]), "titulo_entidade"].replace(ner_tokens_aviso_revogacao)
atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_suspensao_de_licitacao"]), "ner_token"] = atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_suspensao_de_licitacao"]), "titulo_entidade"].replace(ner_tokens_aviso_suspensao_licitacao)

In [53]:
atos_data_df.loc[atos_data_df["ato"].isin(["extrato_de_contrato_ou_convenio"]) & ~atos_data_df["ner_token"].isin(ner_tokens_extrato_contrato.values()), "ner_token"] = ""
atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_aditamento_contratual"]) & ~atos_data_df["ner_token"].isin(ner_tokens_aditamento_contratual.values()), "ner_token"] = ""
atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_licitacao"]) & ~atos_data_df["ner_token"].isin(ner_tokens_aviso_licitacao.values()), "ner_token"] = ""
atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_revogacao_anulacao_de_licitacao"]) & ~atos_data_df["ner_token"].isin(ner_tokens_aviso_revogacao.values()), "ner_token"] = ""
atos_data_df.loc[atos_data_df["ato"].isin(["aviso_de_suspensao_de_licitacao"]) & ~atos_data_df["ner_token"].isin(ner_tokens_aviso_suspensao_licitacao.values()), "ner_token"] = ""

In [54]:
arquivos_erros = atos_data_df.loc[atos_data_df["ner_token"] == "", "arquivo_rast"].unique()

In [55]:
atos_data_df = atos_data_df[~atos_data_df["arquivo_rast"].isin(arquivos_erros)].reset_index(drop=True)

In [56]:
atos_data_text_df = atos_data_text_df[~atos_data_text_df["arquivo_rast"].isin(arquivos_erros)].reset_index(drop=True)

In [57]:
# Filtrar os atos com base em tratamento específico (pegar só aditamento, só extrato, geral)
filt_atos_data_df = specific_parse(atos_data_df)

In [62]:
full_tokenized_texts = get_tokenized_text_lines(filt_atos_data_df, atos_data_text_df)

In [63]:
full_tokenized_texts["treated_text"] = full_tokenized_texts["tokenized_text"].apply(lambda row: " ".join([w.split()[0] for w in row]))

In [64]:
assert full_tokenized_texts["arquivo_rast"].duplicated().any() == False

In [65]:
atos_data_df["ato"].value_counts()

extrato_de_contrato_ou_convenio             4381
aviso_de_aditamento_contratual              1780
aviso_de_licitacao                          1699
aviso_de_revogacao_anulacao_de_licitacao      53
aviso_de_suspensao_de_licitacao               33
Name: ato, dtype: int64

In [66]:
atos_data_text_df["ato"].value_counts()

aviso_de_aditamento_contratual              463
extrato_de_contrato_ou_convenio             442
aviso_de_licitacao                          235
aviso_de_revogacao_anulacao_de_licitacao     16
aviso_de_suspensao_de_licitacao              12
Name: ato, dtype: int64

In [67]:
atos_data_text_df["ato"].value_counts()

aviso_de_aditamento_contratual              463
extrato_de_contrato_ou_convenio             442
aviso_de_licitacao                          235
aviso_de_revogacao_anulacao_de_licitacao     16
aviso_de_suspensao_de_licitacao              12
Name: ato, dtype: int64

In [68]:
full_tokenized_texts = full_tokenized_texts[["arquivo_rast", "tokenized_text"]]

In [70]:
atos_data_df.to_parquet("/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/labelbox_entidades.parquet")
atos_data_text_df.to_parquet("/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/labelbox_atos.parquet")
full_tokenized_texts.to_parquet("/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/tokenized_texts.parquet")

In [71]:
# Separação entre os dados precisa ser feita baseada em textos, e não em palavras
train_set_files = full_tokenized_texts["arquivo_rast"].sample(frac=0.8)
not_train_set_files = full_tokenized_texts.drop(train_set_files.index)

test_set_files = not_train_set_files["arquivo_rast"].sample(frac=0.5)
valid_set_files = not_train_set_files["arquivo_rast"].drop(test_set_files.index)

In [72]:
train_set_df = full_tokenized_texts.loc[train_set_files.index].reset_index(drop=True)
test_set_df = full_tokenized_texts.loc[test_set_files.index].reset_index(drop=True)
valid_set_df = full_tokenized_texts.loc[valid_set_files.index].reset_index(drop=True)

In [73]:
print("VISÃO GERAL DOS CONJUNTOS:")
print("Conjunto de treinamento: {}".format(len(train_set_df)))
print("Conjunto de teste: {}".format(len(test_set_df)))
print("Conjunto de validação: {}".format(len(valid_set_df)))

VISÃO GERAL DOS CONJUNTOS:
Conjunto de treinamento: 934
Conjunto de teste: 117
Conjunto de validação: 117


In [74]:
train_tokenized_texts = list(train_set_df["tokenized_text"])
test_tokenized_texts = list(test_set_df["tokenized_text"])
valid_tokenized_texts = list(valid_set_df["tokenized_text"])

In [77]:
with open(path_samples + "/" + dataset + "_train.txt", "w") as f:
    for group in train_tokenized_texts:
        for line in group:
            f.write(line)
            f.write('\n')
        
        f.write('\n')
    
with open(path_samples + "/" + dataset + "_testa.txt", "w") as f:
    for group in valid_tokenized_texts:
        for line in group:
            f.write(line)
            f.write('\n')
        
        f.write('\n')
    
with open(path_samples + "/" + dataset + "_testb.txt", "w") as f:
    for group in test_tokenized_texts:
        for line in group:
            f.write(line)
            f.write('\n')
        
        f.write('\n')

FileNotFoundError: [Errno 2] No such file or directory: '../atos_revisados/samples_3/contratos_filtrados_v2/./result/_train.txt'

In [85]:
entities = "/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/labelbox_atos.parquet"
texts = "/home/thais/Documentos/Knedle/experiments/members/thais/contrato-licitações/estudo_crf/result/tokenized_texts.parquet"
df1 = pd.read_parquet(entities)
df2 = pd.read_parquet(texts)

df1.head()

,arquivo_rast,text,ato,dodf,treated_text
0,0_aviso_de_licitacao_DODF_010_15-01-2021.txt,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃ...,aviso_de_licitacao,DODF_010_15-01-2021,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 01/202...
1,0_aviso_de_licitacao_DODF_093_20-05-2019.txt,xxbcet AVISO DE LICITAÇÃO xxecet\r\nxxbcet PRE...,aviso_de_licitacao,DODF_093_20-05-2019,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 50/201...
2,2_aviso_de_licitacao_DODF_010_15-01-2021.txt,xxbcet PREGÃO ELETRÔNICO Nº 14/2021 xxecet\nPr...,aviso_de_licitacao,DODF_010_15-01-2021,PREGÃO ELETRÔNICO Nº 14/2021 Processo: 092.041...
3,6_aviso_de_licitacao_DODF_002_05-01-2021.txt,xxbcet AVISO DE LICITAÇÃO xxecet\nxxbcet PREGÃ...,aviso_de_licitacao,DODF_002_05-01-2021,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO (SRP) Nº ...
4,9_extrato_de_contrato_ou_convenio_DODF_004_07-...,xxbcet EXTRATO DE CONTRATO SIMPLIFICADO Nº 6/2...,extrato_de_contrato_ou_convenio,DODF_004_07-01-2016,EXTRATO DE CONTRATO SIMPLIFICADO Nº 6/2015 Esp...
